In [10]:
import pandas as pd
import psycopg2
import math
from queries.ext_query import get_all_notes_for_user
from db_config import host, user, password, db_name
from sqlalchemy import create_engine

id = 1

In [11]:
get_all_notes_for_user = get_all_notes_for_user.format(id)
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}/{db_name}')
connection = engine.connect()
df = pd.read_sql(get_all_notes_for_user, connection, index_col='id')
df2 = pd.read_sql("SELECT * FROM user_agent_to_user_id", connection, index_col='userid') # for future? max_screen_size
connection.close()
df2

,user_agent,max_x,max_y
userid,,,
1,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,1846,948


In [12]:
for let in 'xy':
    df[f'v({let})'] = [abs((df[f'{let}'][index] - df[f'{let}'][index - 1]) / (df['seassion_timestamp'][index] - df['seassion_timestamp'][index -1])) if index > 1 else 0 for index in range(1, len(df[f'{let}']) + 1)]
    df[f'a({let})'] = [abs((df[f'v({let})'][index] - df[f'v({let})'][index- 1]) / (df['seassion_timestamp'][index] - df['seassion_timestamp'][index -1])) if index > 1 else 0 for index in range(1, len(df[f'v({let})']) + 1)]
    df[f'jerk({let})'] = [abs((df[f'a({let})'][index] - df[f'a({let})'][index- 1]) / (df['seassion_timestamp'][index] - df['seassion_timestamp'][index -1])) if index > 1 else 0 for index in range(1, len(df[f'a({let})']) + 1)]

for i in ['v', 'a', 'jerk']:
    df[f'{i}(pixels)'] = [math.sqrt(df[f'{i}(x)'][index] ** 2 + df[f'{i}(y)'][index] ** 2) for index in range(1, len(df[f'a({let})']) + 1)]
    
df['tan'] = [math.tan((df['x'][index] - df['x'][index-1]) / (df['y'][index] - df['y'][index-1] + 0.001)) if index > 1 else 0 for index in range(1, len(df[f'a({let})']) + 1)]

In [13]:
df

,x,y,seassion_timestamp,userid,url,v(x),a(x),jerk(x),v(y),a(y),jerk(y),v(pixels),a(pixels),jerk(pixels),tan
id,,,,,,,,,,,,,,,
1,902,202,5,1,https://edu.gubkin.ru/,0.0,0.00,0.000,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.000000
2,902,202,10,1,https://edu.gubkin.ru/,0.0,0.00,0.000,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.000000
3,845,218,15,1,https://edu.gubkin.ru/,11.4,2.28,0.456,3.2,0.64,0.128,11.840608,2.368122,0.473624,-0.447394
4,845,218,20,1,https://edu.gubkin.ru/,0.0,2.28,0.000,0.0,0.64,0.000,0.000000,2.368122,0.000000,0.000000
5,787,233,25,1,https://edu.gubkin.ru/,11.6,2.32,0.008,3.0,0.60,0.008,11.981653,2.396331,0.011314,-0.885625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5998,1534,873,14985,1,https://edu.gubkin.ru/my/,0.0,0.00,0.000,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.000000
5999,1534,873,14990,1,https://edu.gubkin.ru/my/,0.0,0.00,0.000,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.000000
6000,1534,873,14995,1,https://edu.gubkin.ru/my/,0.0,0.00,0.000,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.000000


In [14]:
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}/{db_name}')
connection = engine.connect()
df.to_sql(name=f"user_{id}", con=connection)
connection.close()